## Code for plotting the Excel data exported from the Li-air Arbin cycler

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string


In [ ]:
file_path = r"C:\Users\lb958\OneDrive - University of Cambridge\Data\Midi\Cell\lb958_CNTS_1625_23523\Channel_19\lb958_CNTS_1497_23523lb958_CNTS_1625_23523_Channel_19.xlsx"
file_name = string.split(file_path, '\\')[-1]
